# Train by seed only

## Loading data

In [ ]:
import pandas as pd

In [ ]:
df_sub = pd.read_csv('../input/ncaam-march-mania-2021/MSampleSubmissionStage1.csv',sep=',',header=0,index_col=None)
df_sub

In [ ]:
df_sub['Season'] = df_sub['ID'].apply(lambda x: x.split('_')[0])
df_sub['Team1'] = df_sub['ID'].apply(lambda x: x.split('_')[1])
df_sub['Team2'] = df_sub['ID'].apply(lambda x: x.split('_')[2])
df_sub

In [ ]:
df_seed = pd.read_csv('../input/ncaam-march-mania-2021/MNCAATourneySeeds.csv',sep=',',header=0,index_col=None)
df_seed.Seed = df_seed.Seed.apply(lambda x: int(x[1:3]))
df_seed

In [ ]:
df_R = pd.read_csv('../input/ncaam-march-mania-2021/MNCAATourneyDetailedResults.csv',sep=',',header=0,index_col=None)
df_R

In [ ]:
df_Rid_1 = df_R[['Season','WTeamID','LTeamID']]
df_Rid_1['Label']=1

df_Rid_0 = df_R[['Season','LTeamID','WTeamID']]
df_Rid_0.columns = ['Season','WTeamID','LTeamID']
df_Rid_0['Label']=0

df_all = pd.concat([df_Rid_1,df_Rid_0])
df_all

In [ ]:
df_seed1 = pd.merge(df_all, df_seed, how='left', left_on=['Season','WTeamID'], right_on = ['Season','TeamID'])
df_seed1.rename(columns={'Seed':'Seed1'}, inplace =True)
df_seed1.drop('TeamID',axis=1, inplace =True)
df_seed1

In [ ]:
df_seed2 = pd.merge(df_seed1, df_seed, how='left', left_on=['Season','LTeamID'], right_on = ['Season','TeamID'])
df_seed2.rename(columns={'Seed':'Seed2'}, inplace =True)
df_seed2.drop('TeamID',axis=1, inplace =True)
df_seed2

In [ ]:
df_seed2['Seed_diff']=df_seed2['Seed1'] - df_seed2['Seed2']
df_seed2

In [ ]:
df_all = df_seed2[['Seed_diff','Label']]
df_all

Xgboost

In [ ]:
import numpy as np
import pandas as pd
import pickle
import xgboost as xgb
from sklearn.model_selection import train_test_split
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

train, test = train_test_split(df_all)

In [ ]:
train_X = train.drop('Label', axis=1).values
train_y = train['Label'].values
test_X = test.drop('Label', axis=1).values
test_y = test['Label'].values


In [ ]:
clf = GridSearchCV(
    estimator = XGBClassifier(use_label_encoder =False,verbosity=0),
    param_grid = {'booster':['gbtree', 'gblinear', 'dart'],
        'n_estimators': [2,5,10, 20, 50],
                  'max_depth': [3,4,5,6,10,20],
                  'subsample':[0.6, 0.7, 0.8]},
    cv=StratifiedKFold(n_splits=5).split(train_X, train_y)
)
clf.fit(train_X, train_y)

In [ ]:
print(clf.best_params_)
print(classification_report(clf.predict(test_X), test_y))

In [ ]:
df_sub['Season']=df_sub['Season'].astype('int64')
df_sub['Team1']=df_sub['Team1'].astype('int64')
df_sub['Team2']=df_sub['Team2'].astype('int64')

In [ ]:
df_sub.info()

In [ ]:
df_sub1 = pd.merge(df_sub, df_seed, how='left', left_on=['Season','Team1'], right_on = ['Season','TeamID'])
df_sub1.rename(columns={'Seed':'Seed1'}, inplace =True)
df_sub1.drop('TeamID',axis=1, inplace =True)
df_sub1

In [ ]:
df_sub2 = pd.merge(df_sub1, df_seed, how='left', left_on=['Season','Team2'], right_on = ['Season','TeamID'])
df_sub2.rename(columns={'Seed':'Seed2'}, inplace =True)
df_sub2.drop('TeamID',axis=1, inplace =True)
df_sub2

In [ ]:
df_sub2['Seed_diff']=df_sub2['Seed1'] - df_sub2['Seed2']
df_sub2

Prediction

In [ ]:
predict_features = df_sub2[['Seed_diff']].values

df_final = df_sub2
df_final['Pred'] = clf.predict_proba(predict_features)[:,1] # get prob of class 1


In [ ]:
df_final

In [ ]:
df_final[['ID','Pred']]
df_final[['ID','Pred']].set_index('ID').to_csv('submission_seed1.csv')

Train with Seed1 and Seed2 and seeddiff

In [ ]:
df_all = df_seed2[['Seed1','Seed2','Seed_diff','Label']]
df_all

train, test = train_test_split(df_all)
train_X = train.drop('Label', axis=1).values
train_y = train['Label'].values
test_X = test.drop('Label', axis=1).values
test_y = test['Label'].values

clf = GridSearchCV(
    estimator = XGBClassifier(use_label_encoder =False,verbosity=0),
    param_grid = {'booster':['gbtree', 'gblinear', 'dart'],
        'n_estimators': [2,5,10, 20, 50],
                  'max_depth': [3,4,5,6,10,20],
                  'subsample':[0.6, 0.7, 0.8]},
    cv=StratifiedKFold(n_splits=5).split(train_X, train_y)
)
clf.fit(train_X, train_y)

predict_features = df_sub2[['Seed1','Seed2','Seed_diff']].values

df_final = df_sub2
df_final['Pred'] = clf.predict_proba(predict_features)[:,1] # get prob of class 1

df_final[['ID','Pred']]
df_final[['ID','Pred']].set_index('ID').to_csv('submission_seed2.csv')